## SVZ cheatsheet
### marusja2 - 2024/2025

In [1]:
import cv2
import numpy as np

# Dokumentace

- help(funkce) - zobrazí nápovědu k funkci
- [Improutils - FIT Gitlab](https://gitlab.fit.cvut.cz/bi-svz/improutils_package)
- [Improutils - Github](https://github.com/ImprolabFIT/improutils)
- [Improutils - Docs](https://improutils.readthedocs.io/en/master/)
- [OpenCV](https://docs.opencv.org/4.x/)

# Výběr snímací soustavy

## Výběr kamery

| Parametr                    | jednotka | označení |
|-----------------------------|----------|----------|
| Rozlišení kamery v pixelech | px       | $r$      |
| Maximální velikost objektu  | mm       | $w$      |
| Přesnost (nejmenší rozdíl)  | mm       | $p$      |

Potřeba zvolit takovou kameru, že platí:

$$r \geq \frac{1.1 \cdot w}{p / 2} = 2.2 \frac{w}{p}$$

- Hodnota $1.1$ v čitateli jako přesah zorného pole (5 % na každé straně)
- Hodnota $2$ ve jmenovateli... alespoň dva pixely pro změnu kontrastu (jeden bílý, jeden černý)

## Příklad

Měření objektů 20 mm a menších, potřeba přesnost 0.01 mm.

$$
\begin{aligned}
w = 20 \text{\ mm}, p = 0.01 \text{\ mm} \ \
    r_{min} = 2.2 \cdot \frac{20}{0.01} = 2.2 \cdot 2000 = 4400 \text{\ px}
\end{aligned}
$$

## Výběr objektivu

Na základě parametrů úlohy $Y, L$ a vybrané kamery $Y'$:

| Parametr                                      | jednotka | označení |
|-----------------------------------------------|----------|----------|
| Maximální velikost objektu                    | mm       | $Y$      |
| Velikost obrazu (delší ze dvou stran)         | mm       | $Y'$     |
| Pracovní vzdálenost (objekt-povrch objektivu) | mm       | $L$      |
| Ohnisková vzdálenost objektivu                | mm       | $f$      |

Potřeba zvolit takový objektiv, že platí:

$$f = Y' \cdot \frac{L}{1.1 \cdot Y}$$

Pokud máme možnou vzdálenost v rozsahu $[L_\min, L_\max]$, potom:

$$Y' \cdot \frac{L_\min}{1.1 \cdot Y} \leq f \leq Y' \cdot \frac{L_\max}{1.1 \cdot Y}$$

Hodnota $1.1$ ve jmenovateli jako přesah zorného pole (5 % na každé straně)

## Příklad

Měření objektů 200 mm a menších, kamera musí být umístěna ve vzdálenosti $[400, 600] \text{\ mm}$. Zvolená kamera má snímací čip o velisosti $7.2 \times 5.4 \text{\ mm}$.

$$
\begin{aligned}
Y = 200 \text{\ mm}, Y' = 7.2 \text{\ mm}, L \in [400, 600] \text{\ mm} \\
f_\min = Y' \cdot \frac{L_\min}{1.1 \cdot Y} = 7.2 \cdot \frac{400}{1.1 \cdot 200} = 13.09 \text{\ mm} \\
f_\max = Y' \cdot \frac{L_\max}{1.1 \cdot Y} = 7.2 \cdot \frac{600}{1.1 \cdot 200} = 19.64 \text{\ mm} \\
\end{aligned}
$$

Můžeme tedy zvolit např. objektiv s $f=16 \text{\ mm}$.

V takovém případě zvolíme pracovní vzdálenost $L = f \cdot \frac{1.1 \cdot Y}{Y'} = 488.8 \text{\ mm}$. Kamera ale bude umístěna dál, a to o délku objektivu!!! Treba jeste pricteme 28mm (dle specsheetu delky objektivu napr).

Pokud i tak nerozhodnome objektiv, tak lze vybrat dle velikosti snimace. (objektiv by mel byt blizko a spis vetsi) 1/1.8" apod.

# Nastavení snímací soustavy
- kamera - rozlišení, barevný rozsah (RGB,mono), velikost senzoru
- objektiv - ohnisková vzdálenost, rozsah clony, velikost objektivu (větší nebo rovna velikosti senzoru), rozsah ostření
- světla

Pracovní vzdálenost (okraj objektivu - povrch objektu)
- vs. minimální pracovní vzdálenost objektivu

Kamera
- rozlišení obrázku (před ořezem)
- -> poměr stran
- barevný rozsah (černobílá/RGB, počet bitů na barvu)
- video... framerate (snímkovací frekvence)
- fyzická velikost senzoru (např. `1/1.8"`, 1" = 16 mm, měřeno na diagonále)
- -> velikost pixelu, mm/px; **přesnost** = dvojnásobek velikosti pixelu

Objektiv
- ohnisková vzdálenost (např. `f = 8 mm`, v průmyslu většinou pevná)
- rozsah clony (např. `F 1.4 - 16.0`), podíl ohniskové vzdálenosti a průměru otvoru clony
- fyzická velikost objektivu (např. `1/1.8"` nebo `7,2 x 5,4 mm`), **větší nebo rovna velikosti senzoru**

Nasvícení
- Směrové... soustředí se přímo na objekt, rovnoběžné paprsky
    - nereflexivní povrchy
    - **zvýraznění reliéfu**
- Difúzní/rozptýlené světlo... nepřímé
    - lesklé povrchy
    - často jako diuzor přes klasické (přímé) světlo
    - **skrytí reliéfu**
- Zadní... použití pro maximální **zvýraznění obrysu**
    - se silným difuzorem
    - Nejčastěji se používá ke zjišťování přítomnosti/nepřítomnosti otvorů/mezer, ke zjištění orientace objektů, či k jejich měření
- Dark Field... velké množství LED diod okolo objektu, svítí pod ostrým bočním úhlem
    - **velké zvýraznění reliéfu**, vyrytých/vytlačených nápisů atd.
- Kopulové... Opak dark fieldu, světlo přichází ze všech stran, rovnoměrně
    - **maximální skrytí reliéfu**
- *Koaxiální*... díky propustnému zrcadlu světlo ze stejného směru, jako kamera
    - DOAL = Diffused On Axis Light
    - elimance odlesků, zvýraznění detailů

### Vinětace
tmavé rohy obrázku, do rohů senzoru nedopadá dostatek světla (oproti středu)

Příčiny:
- konstrukce objektivu (příliš úzký)
- příliš **otevřená** clona

Řešení:
- vyměnit objektiv za širší
- uzavřít clonu

### Chromatická aberace
barevné lemování hran

Příčiny:
- konstrukce čočky objektivu
- příliš **otevřená** clona

Řešení:
- vyměnit objektiv (jiný materiál čočky, menší zoom)
- uzavřít clonu

### Difrakce
snížení ostrosti obrazu, zrnitost
- obecný problém průchodu vlnění úzkou štěrbinou
- [Difrakce (Wikipedia)](https://cs.wikipedia.org/wiki/Difrakce)

Příčiny:
- příliš **uzavřená** clona

Řešení:
- nastavit clonu na **sweet-spot** (omezení všech vad optiky)

### Distorze
Zakřivení čar, které jsou v realitě rovné
- zejména u **širokoúhlých objektivů**
- **radiální** distorze... barrel/pincushion
- **tangenciální**... "naklonění" obrazu, jedna strana blíže než druhá

Řešení:
- výměna objektivu
- digitální **kalibrace** obrazu

Kalibrace:
- na základě snímků referenčního obrazu (šachovnice známých rozměrů)
- ztrátová... odříznutí zakřiveného obrazu v krajích
- formálně... nalezení kalibračních parametrů
- $k_{1:3}$ pro radiální
- $p_{1:2}$ pro tangenciální

# Kalibrace kamery
- Nafotit 10-20 fotek šachovnice z různých úhlů a v různých částech obrazu.
- Zkalibrovat.


- Velikost šachovnice: Velikost šachovnice by měla být zvolena tak, aby při požadované pracovní vzdálenosti zabírala alespoň 50% snímku při pohledu, kdy je šachovnice paralelně se snímačem (fronto-paralelně).
- Natočení vzoru: Pro zjištění distorzních parametrů by měly postačit pouze fronto-paralelní snímky šachovnice. Pro zjištění vnitřních parametrů je zapotřebí vzor natáčet v různých úhlech. Doporučené natočení je +- 45° okolo vertikální a horizontální osy. I v případě, že chceme zjistit pouze distorzní parametry, je vhodné natáčet vzor v různých úhlech a vytvořit větší dataset.
- Rozložení snímků: Musíme vzor umístit do všech částí snímku. Pokud nebudeme mít např. vzor na okrajích, parametry nebudou dostatečně svázány (constrained).
- Filtrace snímků: Po samotné kalibrací je vhodné provést filtraci snímků. Často nekvalitní snímky mohou zhoršit výsledky kalibrace a jejich reproječní chyba je vyšší než u ostatních snímků. Následně je možné pořídit snímek znovu a opětovně provést kalibraci.
- Overfitting: Nízká reprojekční chyba neznamená nutně dobrou kalibraci. Může se jednat o přeučení (overfitting) modelu na daný dataset. Nastává při použití příliš flexibilního modelu.

In [2]:
from improutils import create_file_path, reindex_image_files, camera_calib
import yaml

def calib1():
    calib_folder_path = "data/calibration"

    reindex_image_files(calib_folder_path)
    images_format = '%01d.bmp'

    calibration_file_name = "calibration.yaml" ### *.yaml
    output_calib_file_path = create_file_path(calib_folder_path, calibration_file_name)

    chess_shape = (9, 6)  # počet rohů mezi čtverci šachovnice, ignoruje jednu řadu čtverců od každého kraje

    input_source = create_file_path(calib_folder_path, images_format)
    camera_matrix, dist_coefs, good_images = camera_calib(input_source=input_source, chess_shape=chess_shape,output_calib_file=output_calib_file_path) ###

In [3]:
def correct_frame(frame, camera_matrix, dist_coeffs):
    """Returns undistorted frame."""
    return cv2.undistort(frame, camera_matrix, dist_coeffs)

# Měření dílu
- Nafotit díl (s referenčním objektem)
- Změřit díl


Relevatntní funkce:
- `segmentation_two_thresholds`
- `segmentation_auto_threshold`
- `segmentation_adaptive_threshold`
- `segmentation_auto_threshold`

Hledání kontury:
- `cv2.dilate` [doc](https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html)
- `cv2.erode`
- `fill_holes`
- `find_contours` -> `(contour_drawn: np.ndarray, count: int, contours: list)`

`enum_contours` (hw01) !

Převod na geometrický útvar:
- `cv2.minAreaRect` -> `(center(x, y), (width, height), angle of rotation)` [guide](https://theailearner.com/tag/cv2-minarearect/)
- `cv2.boundingRect` kolmý na osy x, y
- `cv2.minEnclosingCircle` -> `(center, radius)`

In [4]:
import improutils
from improutils import apply_mask
from ipywidgets import interact, interactive, fixed, interact_manual  # slidery na segmentaci

def slider():
    # Grayscale/na jedné barvě
    img = load_image('images/basic.png')
    @interact(threshold_range=create_slider(min=0, max=255, description='Threshold range:',))
    def _(threshold_range):
        mask = improutils.segmentation_two_thresholds(img, threshold_range[0], threshold_range[1])
        plot_images(mask)
    # HSV
    @interact(h_range=create_slider(min=0, max=360, description='Hue:'),
              s_range=create_slider(min=0, max=255, description='Saturation:'),
              v_range=create_slider(min=0, max=255, description='Value:'))
    def _(h_range, s_range, v_range):

        lower_bound = (improutils.to_intensity(h_range[0]), s_range[0], v_range[0])
        upper_bound = (improutils.to_intensity(h_range[1]), s_range[1], v_range[1])

        mask = improutils.segmentation_two_thresholds(img, lower_bound, upper_bound)
        plot_images(mask, apply_mask(img, mask))

# Freestyle

### General
#### Popisné charakteristiky
def form_factor(contour):
def roundness(contour):
def aspect_ratio(contour):
def convexity(contour):
def solidity(contour):
def compactness(contour):
def extent(contour):

- Formfactor (špičatost)
- Roundness (kulatost)
- Aspect Ratio (poměr stran)
- Convexity (konvexita, vypouklost)
- Solidity (plnost, celistvost)
- Compactness (kompaktnost, hutnost)
- Extent (dosah, rozměrnost)


### Some usage examples

### Perspektiva

### Helper functions

In [5]:
def describe_img(img: np.ndarray) -> None:
    resolution = img.shape[:2] ###
    print(f'Rozlišení obrazu:          {img.shape[:2] = }')

    number_of_channels = img.shape[2] ###
    print(f'Počet kanálů:              {img.shape[2] = }')

    # np.min == np.amin != np.maximum
    print(f'Nejnižší hodnota v obrazu: {np.min(img) = }')
    print(f'Nevyšší hodnota v obrazu:  {np.max(img) = }')
    print(f'Průměrná hodnota obrazu:   {np.mean(img) = }')

    print(f'Rozlišení v MPix:          {resolution[0] * resolution[1] / (10 ** 6) = }')

In [6]:
def describe_contour(contour: np.ndarray) -> None:
    area = cv2.contourArea(contour) ###
    print(f'Obsah kontury:             {area = }')

    perimeter = cv2.arcLength(contour, True) ###
    print(f'Obvod kontury:             {perimeter = }')

    rect = cv2.minAreaRect(contour)
    shape_width, shape_height = rect[1]

    center = cv2.moments(contour) ###
    center_x = int(center["m10"] / center["m00"])
    center_y = int(center["m01"] / center["m00"])
    print(f'Střed kontury:             {(center_x, center_y) = }')

In [7]:
def angle_lines(line1, line2):
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2

    angle1 = math.atan2(y1 - y2, x1 - x2)
    angle2 = math.atan2(y3 - y4, x3 - x4)

    return math.degrees(angle1 - angle2)

### HW01

In [8]:
import math
import itertools
from improutils import to_intensity, segmentation_two_thresholds, find_contours, load_image, crop, to_hsv, plot_images


def hw01():
    # načtení obrazu
    img = load_image('images/basic.png')
    img = crop(img, 440, 200, 1600, 900)
    img_hsv = to_hsv(img)
    plot_images(img,img_hsv)
    # segmentace obdélníků
    lower_bound = (to_intensity(18), 140, 149)
    upper_bound = (to_intensity(64), 255, 201)

    lower_bound_other = (to_intensity(314), 113, 0)
    upper_bound_other = (to_intensity(360), 255, 255)

    rect_others = segmentation_two_thresholds(img_hsv, lower_bound_other, upper_bound_other) ### prahy pro segmentaci v RGB
    rect_ref = segmentation_two_thresholds(img_hsv, lower_bound, upper_bound) ### prahy pro segmentaci v RGB
    rect_mask = cv2.add(rect_others, rect_ref)

    # nalezení referenčního obdélníku podle velikosti kontury
    drawn_ref, _, ref_cnt = find_contours(rect_mask, 30000, 85000)

    # nalezení kontur všech obdélníků
    drawn_ref_others, _, ref_cnt_others = find_contours(rect_mask, 10000)

    ref_width_real = 40
    ref_height_real = 80

    # vypočtení poměru mm/pix
    rect = cv2.minAreaRect(ref_cnt[0])
    ref_width_image, ref_height_image = rect[1]
    real_image_ratio = min(ref_width_real, ref_height_real) / min(ref_width_image, ref_height_image)

    print(f'Recalculated size: {(ref_width_image*real_image_ratio, ref_height_image*real_image_ratio)}')
    print(f'Ratio between real width and image width: {real_image_ratio}')


    def get_bounding_rect_center(contour):
        x, y, w, h = cv2.boundingRect(contour)
        center_x = x + w // 2
        center_y = y + h // 2
        return (center_x, center_y)
    contours = ref_cnt_others
    contours = sorted(contours, key=lambda c: (get_bounding_rect_center(c)[0], get_bounding_rect_center(c)[1]))

    contour_images = []

    for contour in contours:
        contour_images.append(img.copy())
        cv2.drawContours(contour_images[-1], [contour], -1, color=(0, 255, 0 ), thickness=5)

    plot_images(*contour_images,titles=[0,1,2,3,4],title_size=64)

    index_list = list(range(len(contours)))
    combinations = list(itertools.combinations(index_list , 2))

    def line_segment_to_point_dist(l_pt1, l_pt2, dst_pt,ret_points=False):
        x1, y1 = l_pt1
        x2, y2 = l_pt2
        x0, y0 = dst_pt

        # Umocněná velikost úsečky
        line_len_sq = (x2 - x1) ** 2 + (y2 - y1) ** 2

        # Projekční faktor
        t = ((x0 - x1) * (x2 - x1) + (y0 - y1) * (y2 - y1)) / line_len_sq

        # Určíme vzdálenosti od bodu [x0,y0] vůči různým situacím
        if t < 0:
            # Vzdálenost od bodu [x1,y1]
            proj_x = x1
            proj_y = y1
        elif t > 1:
            # Vzdálenost od bodu [x2,y2]
            proj_x = x2
            proj_y = y2
        else:
            # Vzdálenost kolmice na úsečku
            proj_x = x1 + t * (x2 - x1)
            proj_y = y1 + t * (y2 - y1)

        if ret_points:
            return math.sqrt((x0 - proj_x) ** 2 + (y0 - proj_y) ** 2), [(x0,y0), [proj_x, proj_y]]
        # Spočteme vzdálenost
        return math.sqrt((x0 - proj_x) ** 2 + (y0 - proj_y) ** 2)

    def line_segments_dist(l1_pt1, l1_pt2, l2_pt1, l2_pt2, ret_points = False):
        # Vždy 2 možnosti bodu pro jednu úsečku
        distances = [
            line_segment_to_point_dist(l1_pt1, l1_pt2, l2_pt1,ret_points),
            line_segment_to_point_dist(l1_pt1, l1_pt2, l2_pt2,ret_points),
            line_segment_to_point_dist(l2_pt1, l2_pt2, l1_pt1,ret_points),
            line_segment_to_point_dist(l2_pt1, l2_pt2, l1_pt2,ret_points)
        ]

        if not ret_points:
            return min(distances)

        min_distance, closest_points = min(distances, key=lambda x: x[0])

        return min_distance, closest_points

    def rect_dist(r1_pts, r2_pts, ret_points=False):
        # Obdélník je tvořen 4 úsečkami
        r1_segments = [
            (r1_pts[0], r1_pts[1]),
            (r1_pts[1], r1_pts[2]),
            (r1_pts[2], r1_pts[3]),
            (r1_pts[3], r1_pts[0])
        ]

        r2_segments = [
            (r2_pts[0], r2_pts[1]),
            (r2_pts[1], r2_pts[2]),
            (r2_pts[2], r2_pts[3]),
            (r2_pts[3], r2_pts[0])
        ]
        closest_pts = None

        # Najdeme nejmenší vzdálenost mezi libovolnou úsečkou r1 a libovolnou úsečkou r2
        min_distance = float('inf')
        for seg1 in r1_segments:
            for seg2 in r2_segments:
                dist = line_segments_dist(seg1[0], seg1[1], seg2[0], seg2[1],ret_points)
                if ret_points and dist[0] < min_distance:
                    closest_pts = dist[1]
                    min_distance = dist[0]
                elif not ret_points and dist < min_distance:
                    min_distance = dist

        if ret_points:
            return min_distance, closest_pts
        else:
            return min_distance

    rectangles = [cv2.boxPoints(cv2.minAreaRect(cnt)) for cnt in contours]

    for comb in combinations:
        idx1, idx2 = comb

        r1_pts = rectangles[idx1]
        r2_pts = rectangles[idx2]

        dist_px = rect_dist(r1_pts, r2_pts)

        print(f"{idx1} <-> {idx2}: {dist_px * real_image_ratio / 10:.2f} cm")


    image = img.copy()

    for comb in combinations:
        idx1, idx2 = comb
        r1_pts = rectangles[idx1]
        r2_pts = rectangles[idx2]

        dist_px, (pt1, pt2) = rect_dist(r1_pts, r2_pts, True)

        pt1, pt2 = map(tuple, map(lambda pt: map(int, pt), (pt1, pt2)))

        dist_cm = dist_px * real_image_ratio / 10
        cv2.line(image, pt1, pt2, color=(255, 0, 0), thickness=2)

    # Rozdělené abych text vypsal přes čáry
    for comb in combinations:
        idx1, idx2 = comb
        r1_pts = rectangles[idx1]
        r2_pts = rectangles[idx2]

        dist_px, (pt1, pt2) = rect_dist(r1_pts, r2_pts, True)

        pt1, pt2 = map(tuple, map(lambda pt: map(int, pt), (pt1, pt2)))
        position = tuple(map(int, ((pt1[0] + pt2[0]) / 2 - 30, (pt1[1] + pt2[1]) / 2 - 20)))

        dist_cm = dist_px * real_image_ratio / 10
        cv2.putText(image, f'{dist_cm:.2f}cm'.format(".2f"), position, cv2.FONT_HERSHEY_COMPLEX, 0.65, (255, 255, 255), 2, cv2.LINE_AA)

## library.ipynb

In [9]:
def rotate_image(image, angle, image_center=None):
    """ Rotates the input image by specified angle.

    Parameters
    ----------
    image : np.ndarray
        Image to be rotated.
    angle : float
        Rotation angle.
    image_center : Optional[tuple(int, int)]
        Center of rotation.
    Returns
    -------
    np.ndarray
        Returns the rotated input image by specified angle.
    """
    if image_center is None:
        image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

In [10]:
from improutils import copy_to


def draw_rotated_text(img, text, point, angle, text_scale, text_color, text_thickness):
    img_filled = np.full(img.shape, text_color, dtype=np.uint8)
    # create rotated text mask
    text_mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    cv2.putText(text_mask, "{:.2f} cm".format(text), point, 0, text_scale, (255, 255, 255), text_thickness)
    if angle > 0:
        angle = -angle + 90
    elif angle < 0:
        angle = angle + 90
    text_mask = rotate_image(text_mask, -angle, point)
    result = copy_to(img_filled, img.copy(), text_mask)
    return result

In [11]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import ipywidgets as widgets
from IPython.display import display

def create_slider(min, max, description):
    description = description.ljust(30, '\xa0')
    return widgets.IntRangeSlider( min=min, max=max, step=1,value=[min,max],
                                   description=description,
                                   continuous_update=False,
                                   orientation='horizontal',
                                   style=dict(description_width='initial'),
                                   layout=widgets.Layout(width='auto'),
                                   )

def multicolor_segmentation(func,colors):
    """ Allows interactive HSV thresholding for multiple colors with saving and returning thresholds that are picked by the user.

    Parameters
    ----------
    func : function
        function with arguments hue = h_range (int, range: 0-360), saturation = s_range (int, range: 0-255), value = v_range (int, range: 0-255)
    colors : list
        list of colors that the user can choose from, e.g. ['red', 'green', 'blue'], these colors will be used as keys in the output dictionary
    Returns
    -------
    color_thresholds: dict
        Returns a dictionary with the chosen thresholds for each color, e.g. {'red': (0, 0, 0), 'green': (0, 0, 0), 'blue': (0, 0, 0)}, can be also empty if no thresholds were saved
    """
    color_thresholds = {}

    # initialize sliders, buttons etc.
    h_slider=create_slider(min=0, max=360, description='Hue:')
    s_slider=create_slider(min=0, max=255, description='Saturation:')
    v_slider=create_slider(min=0, max=255, description='Value:')

    color_dropdown = widgets.Dropdown(options=colors, description='Color:'.ljust(30, '\xa0'), style ={'description_width': 'initial'},layout = {'width': 'max-content'})

    save_button = widgets.Button(description='Save threshold for color',layout=widgets.Layout(width='auto'),button_style='success')
    finish_button = widgets.Button(description='Return saved thresholds',layout=widgets.Layout(width='auto'),button_style='danger')

    text_output = widgets.Output()
    interactive_output = widgets.interactive_output(func,{'h_range':h_slider,'s_range':s_slider,'v_range':v_slider})

    # widget layout
    input_box = widgets.VBox([h_slider,s_slider,v_slider,color_dropdown])
    button_box = widgets.HBox([save_button, finish_button])
    other_box = widgets.VBox([text_output, interactive_output])

    def reset_sliders():
        h_slider.value = (0,360)
        s_slider.value = (0,255)
        v_slider.value = (0,255)

    # button callbacks
    def on_save_clicked(b):
        with text_output:
            text_output.clear_output()
            color_thresholds[color_dropdown.value] = (h_slider.value, s_slider.value, v_slider.value)
            print(f"Saved for color '{color_dropdown.value}', threshold: {color_thresholds[color_dropdown.value]}\nResetting sliders...\nChanging to next color...")
            reset_sliders()
            # set next color in dropdown
            color_dropdown.value = colors[(colors.index(color_dropdown.value)+1)%len(colors)]


    def on_finish_clicked(b):
        with text_output:
            text_output.clear_output()
            print('Returned saved thresholds!')
            reset_sliders()


    save_button.on_click(on_save_clicked)
    finish_button.on_click(on_finish_clicked)
    # display widget
    display(input_box, button_box,other_box)

    return color_thresholds

## Extra

## library.py

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Union
from pathlib import Path
from prettytable import PrettyTable
from improutils import *

def camera_calibration(calib_path: str,
                       chess_shape: Tuple[int,int],
                       cv2_flags:int = 0,
                       extensions: List[str] = ["jpg", "jpeg" ,"png", "tiff", "bmp"]) -> Tuple[float,
np.ndarray,
np.ndarray,
Tuple[np.ndarray],
Tuple[np.ndarray],
np.ndarray,
np.ndarray,
np.ndarray,
Dict[str,np.ndarray]]:
    """Calibrates camera from images with chessboard pattern, using OpenCV's cv2.calibrateCameraExtended function

    Args:
        calib_path (str): path to the folder containing chessboard pattern images
        chess_shape (Tuple[int,int]): interior corner count in the format of rows, columns
        cv2_flags (int, optional): additional OpenCV's flags for cv2.calibrateCameraExtended. Defaults to 0.
        extensions (List[str], optional): allowed image extensions. Defaults to ["jpg", "jpeg" ,"png", "tiff"].

    Raises:
        ValueError: if calibration images have different sizes
        ValueError: if no calibration images were found or could not be read from the provided path
        ValueError: if no chessboard patterns were detected in the images

    Returns:
        Tuple[float, np.ndarray, np.ndarray, Tuple[np.ndarray], Tuple[np.ndarray], np.ndarray, np.ndarray, np.ndarray, Dict[str,np.ndarray]]:
        returns the output from cv2.calibrateCameraExtended and dictionary with image names as keys and images with drawn chessboard corners as values
    """
    print(f"Processing images from {calib_path} with possible extensions {extensions}")
    def correct_extension(path, extensions):
        return path.is_file() and path.suffix[1:].lower() in extensions
    # termination criteria for subpixel corner detection
    # by default it is set to 30 iterations and epsilon = 0.001
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((chess_shape[0] * chess_shape[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:chess_shape[0], 0:chess_shape[1]].T.reshape(-1, 2)

    # arrays to store object points and image points from all the images.
    objpoints = [] # 3D point in real world space
    imgpoints = [] # 2D points in image plane.

    image_paths = sorted([path for path in Path(calib_path).glob("*") if correct_extension(path,extensions)])
    chess_brd_images = 0
    read_images = 0
    chessboard_images = {}
    img_size = None
    for img_path in image_paths:
        img_name = img_path.name

        img = cv2.imread(img_path)
        if img is None:
            print(f"File {img_name} could not be read, skipping...")
            continue
        else:
            read_images += 1
            if img_size is None:
                # need to be in the format of width, height
                img_size = img.shape[:2][::-1]
            else:
                if img_size != img.shape[:2][::-1]:
                    raise ValueError("All images must have same size.")
            print(f"File {img_name} is being processed...")

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, chess_shape, None)

        # if found, add object points, image points (after refining them)
        if ret:
            chess_brd_images += 1
            print(f"\t Corners found!")
            objpoints.append(objp)
            subpix_corners = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            imgpoints.append(subpix_corners)

            chessboard_images[img_name] = cv2.drawChessboardCorners(img, chess_shape, subpix_corners, ret)

        else:
            print(f"\t Corners NOT found!")
            continue

    print(f"Number of images with detected chessboard: {chess_brd_images}/{read_images}")

    if read_images == 0:
        raise ValueError("No images were read from the provided path.")

    if chess_brd_images == 0:
        raise ValueError("No chessboard patterns were detected in the images.")

    calib_values = cv2.calibrateCameraExtended(objpoints, imgpoints, img_size, cameraMatrix=None, distCoeffs=None, flags=cv2_flags, criteria=criteria)
    reprojection_error, camera_matrix, dist_coeffs, rvecs, tvecs, std_deviations_intrinsics, std_deviations_extrinsics, per_view_errors = calib_values
    return reprojection_error, camera_matrix, dist_coeffs, rvecs, tvecs, std_deviations_intrinsics, std_deviations_extrinsics, per_view_errors, chessboard_images

def calibration_stats(reprojection_error:float,
                      camera_matrix: np.ndarray,
                      dist_coeffs:np.ndarray,
                      std_deviations_intrinsics:np.ndarray=None,
                      per_view_errors:np.ndarray=None,
                      view_names:List[str]=None,
                      pixel_size:Union[float,Tuple[float,float]]=None) -> None:
    """Prints calibration statistics using.
    RMS re-projection error, estimated intrinsics and distortion parameters, standard deviations of intrinsics, focal length in millimeters and per view reprojection errors.

    Args:
        reprojection_error (float): re-projection error from cv2.calibrateCamera
        camera_matrix (np.ndarray): camera matrix from cv2.calibrate
        dist_coeffs (np.ndarray): distortion coefficients from cv2.calibrateCamera
        std_deviations_intrinsics (np.ndarray, optional): std_deviations_intrinsics from cv2.calibrateCameraExtended. Defaults to None.
        per_view_errors (np.ndarray, optional): per_view_errors from cv2.calibrateCameraExtended. Defaults to None.
        view_names (List[str], optional): image names for which we detected the chessboard. Defaults to None.
        pixel_size (Union[float,Tuple[float,float]], optional): size of physical pixels of a camera in micrometers eg. 4.8 or 5.86 or [5.86, 4.8] for non square pixels. Defaults to None.
    """
    # opencv always returns atleast 4 distortion coefficients
    params_amount = 4 + dist_coeffs.shape[1]

    parameters = ["fx", "fy", "cx", "cy", "k1", "k2", "p1", "p2", "k3", "k4", "k5", "k6", "s1", "s2", "s3", "s4", "Tx", "Ty"]
    units = ["pixels"] * 4 + ["unitless"] * (params_amount - 4)

    print(f"RMS re-projection error: {reprojection_error:.5f} pixels")

    print(f"\nEstimated intrinsics parameters")
    intrinsics_table = PrettyTable()
    intrinsics_table.add_column("Parameter", parameters[:4])
    intrinsics_table.add_column("Estimated Value", [f"{val:.5f}" for val in [camera_matrix[0, 0], camera_matrix[1, 1], camera_matrix[0, 2], camera_matrix[1, 2]]])
    intrinsics_table.add_column("Unit", units[:4])
    print(intrinsics_table)

    print(f"\nEstimated Distortion parameters")
    distortion_table = PrettyTable()
    distortion_table.add_column("Parameter", parameters[4:params_amount])
    distortion_table.add_column("Distortion", [f"{val:.5f}" for val in dist_coeffs[0, :params_amount-4]])
    distortion_table.add_column("Unit", units[4:params_amount])
    print(distortion_table)

    if std_deviations_intrinsics is not None:
        print(f"\nIntrinsic parameters standard deviation")
        intrinsics_std_table = PrettyTable()
        intrinsics_std_table.add_column("Parameter", parameters[:params_amount])
        intrinsics_std_table.add_column("Value", [f"±{val:.5f}" for val in std_deviations_intrinsics[:params_amount,0]])
        intrinsics_std_table.add_column("Unit", units[:params_amount])
        print(intrinsics_std_table)

    if pixel_size is not None and std_deviations_intrinsics is not None:
        if not isinstance(pixel_size, tuple):
            pixel_size = (pixel_size, pixel_size)
        print(f"\nEstimated Focal length in millimeters")
        focal_length_table = PrettyTable()
        focal_length_table.add_column("Parameter", parameters[:2])
        focal_length_table.add_column("Value ± Std Deviation", [f"{val*pix_size/1000:.5f} ± {std*pix_size/1000:.5f}" for val, pix_size, std in zip([camera_matrix[0, 0], camera_matrix[1, 1]], pixel_size, std_deviations_intrinsics[:2,0])])
        focal_length_table.add_column("Unit", ["millimeter"] * 2)
        print(focal_length_table)

    if per_view_errors is not None:
        print(f"\nPer view reprojection errors")
        view_error_table = PrettyTable()
        # Sort the view names and errors by the errors in descending order
        sorted_views_and_errors = sorted(zip(view_names, per_view_errors[:,0]), key=lambda x: x[1], reverse=True)
        sorted_view_names, sorted_errors = zip(*sorted_views_and_errors)
        view_error_table.add_column("Image name", sorted_view_names)
        view_error_table.add_column("Re-projection error (sorted)", [f"{val:.5f}" for val in sorted_errors])
        view_error_table.add_column("Unit", ["pixels"] * len(sorted_view_names))
        print(view_error_table)

def correct_frame(img, camera_matrix, dist_coeffs):
    """Returns undistorted image."""
    return cv2.undistort(img, camera_matrix, dist_coeffs)

def _plot_grid(xv, yv, squares, ax):
    for i  in np.linspace(0, xv.shape[1] - 1, squares+1, dtype=int):
        ax.plot(xv[i,:], yv[i,:], 'k-')
    for j in np.linspace(0, xv.shape[0] - 1, squares+1, dtype=int):
        ax.plot(xv[:,j], yv[:,j], 'k-')

    ax.axis('off')

def _radial_distortion(xv, yv, k):
    xv_radial = np.zeros_like(xv)
    yv_radial = np.zeros_like(yv)
    for i in range(xv.shape[0]):
        for j in range(xv.shape[1]):
            r = np.sqrt(xv[i,j]**2 + yv[i,j]**2)
            radial = (1 + (k[0]*(r**2) + k[1]*(r**4) + k[2]*(r**6)))/(1 + (k[3]*(r**2) + k[4]*(r**4) + k[5]*(r**6)))
            xv_radial[i,j] = xv[i,j]*radial
            yv_radial[i,j] = yv[i,j]*radial
    return xv_radial, yv_radial

def _tangetial_distortion(xv, yv, p):
    xv_tang = np.zeros_like(xv)
    yv_tang = np.zeros_like(yv)
    for i in range(xv.shape[0]):
        for j in range(xv.shape[1]):
            x = xv[i,j]
            y = yv[i,j]
            r = np.sqrt(x**2 + y**2)
            x_tang = x + (2*p[0]*x*y + p[1]*(r**2 + 2*x**2))
            y_tang = y + (p[0]*(r**2 + 2*y**2) + 2*p[1]*x*y)
            xv_tang[i,j] = x_tang
            yv_tang[i,j] = y_tang
    return xv_tang, yv_tang

def plot_distortion(k1:float,k2:float,k3:float,k4:float,k5:float,k6:float, p1:float,p2:float) -> None:
    """Plots radial, tangential and compounded (radial + tangential) distortion grid. Using the Brown-Conrady model.

    Args:
        k1 (float): radial distortion coefficient
        k2 (float): radial distortion coefficient
        k3 (float): radial distortion coefficient
        k4 (float): radial distortion coefficient
        k5 (float): radial distortion coefficient
        k6 (float): radial distortion coefficient
        p1 (float): tangential distortion coefficient
        p2 (float): tangential distortion coefficient
    """
    k = (k1,k2,k3,k4,k5,k6)
    p = (p1,p2)
    squares = 10 # amount of squares in the grid
    pts = 100
    # realistical values for image with 2500 x 2500 pixels with focal length of 35mm which is close to 10500 pixels with basler camera pixel size, origin is in the center - therfore x and y should be within values +-(2500/10500)/2
    width = 0.23
    height = 0.23
    xv, yv = np.meshgrid(np.linspace(-width/2,width/2,pts), np.linspace(-height/2,height/2,pts))

    xv_radial, yv_radial = _radial_distortion(xv, yv, k)
    xv_tang, yv_tang = _tangetial_distortion(xv, yv, p)

    _, axs = plt.subplots(1, 3, figsize=(15, 5))

    _plot_grid(xv_radial, yv_radial, squares, axs[0])
    axs[0].set_title('Radial distortion grid')

    _plot_grid(xv_tang, yv_tang, squares, axs[1])
    axs[1].set_title('Tangential distortion grid')

    _plot_grid(xv_radial + xv_tang, yv_radial + yv_tang, squares, axs[2])
    axs[2].set_title('Compounded distortion grid')
    plt.show()
